# crawl contact info

因為抓的時候，後來清理資料才發現 API 裡面沒有張貼者的聯絡資訊，  
所以後來才再補爬。

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import random
import re
import json
import os

In [2]:
path_root = os.path.join(os.getcwd(), "../")
path_result = os.path.join(path_root, "./result")

if not os.path.exists(path_result):
    os.mkdir(path_result)

## session

## crawl 20210614

改版後目前這個就失效了

## crawl 20210621

用 selenium 硬爬

In [6]:
from selenium import webdriver

In [7]:
meta_refresh = pd.read_csv('../result/meta_refresh.csv')
postids = list(meta_refresh['id'])
len(postids)

15267

In [8]:
path_driver = "C:\InstallSoftware\crawler\chromedriver.exe"
browser = webdriver.Chrome(executable_path = path_driver)

In [9]:
%%time

contact = []
# for idx in range(len(postids)): 
for idx in range(0,1500): 
    if (idx % 10) == 0: print(idx)
    
    url = f'https://rent.591.com.tw/home/{postids[idx]}'
    browser.get(url)
    
    try:
        tmp_contact = browser.find_element_by_xpath("//div[@id='rightConFixed']//span[@class='tel-txt']").text
        tmp_contact = re.search('[0-9-]+', tmp_contact).group(0)
        result = [{'post_id': postids[idx], 'contact': tmp_contact}]
    except:
        result = [{'post_id': postids[idx], 'contact': None}]
    
    contact = contact + result
    browser.implicitly_wait(1.5)
    sleeptime = random.uniform(0, 1)
    time.sleep(sleeptime)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
Wall time: 33min 35s


In [10]:
# print(browser.page_source)
browser.quit()

## save

In [77]:
path_json = os.path.join(path_result, './contact.json')
with open(path_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(contact, jsonfile, ensure_ascii = False)

---

## update data to mongoDB


In [78]:
import os
import sys
import numpy as np
import json
import re
from pymongo import MongoClient

In [79]:
path_root = os.path.join(os.getcwd(), "../")
path_key = os.path.join(path_root, './key/key.txt')
path_result = os.path.join(path_root, "./result")
json_contact = os.path.join(path_result, "./contact.json")

In [80]:
conn_string = open(path_key, 'r').read()

### read data

In [81]:
with open(json_contact, "r", encoding='utf-8') as jsonfile:
    raw_contact = json.load(jsonfile)

In [83]:
for i in range(len(raw_contact)):
    try:
        a = re.search('[0-9-]+ ?', raw_contact[i]['contact'])
        raw_contact[i]['contact'] = a.group(0).strip().replace('-', '')
    except:
        next

### upload

In [90]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']
# collection = db['meta_data']
# collection = db['cathay_search']
collection = db['my_search']

In [91]:
%%time

for i in range(len(raw_contact)):
    collection.update_one({'post_id': raw_contact[i]['post_id']}, {'$set': {'contact': raw_contact[i]['contact']}})

Wall time: 7min 47s


In [92]:
cluster.close()

## END